In [4]:
import pandas as pd
from covidb.pull import data_pull_csv
from covidb.process import get_todays_data

In [5]:
# print("Starting Pull")
# data_pull_csv()
# print("Finished Pull")
print("Started Processing")
data_dict = get_todays_data()
print("Finished Processing")

Started Processing
Finished Processing


In [6]:
data_dict["today"]

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Recovered,Deaths,is_full_state,county,state,confirmed_yesterday,confirmed_delta_today
0,Washington,US,47.4009,-121.4905,2020-03-16,904,1,48,True,NaN,Washington,643,261
1,New York,US,42.1657,-74.9481,2020-03-16,967,0,10,True,NaN,New York,732,235
2,California,US,36.1162,-119.6816,2020-03-16,557,6,7,True,NaN,California,426,131
3,New Jersey,US,40.2989,-74.5210,2020-03-16,178,1,2,True,NaN,New Jersey,98,80
4,Diamond Princess,US,35.4437,139.6380,2020-03-16,47,0,0,False,NaN,NaN,0,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,"Riverside County, CA",US,33.9533,-117.3961,2020-03-16,0,0,0,False,Riverside County,California,0,0
243,"Shelby County, TN",US,35.1269,-89.9253,2020-03-16,0,0,0,False,Shelby County,Tennessee,0,0
244,"St. Louis County, MO",US,38.6103,-90.4125,2020-03-16,0,0,0,False,St. Louis County,Missouri,0,0
245,"Suffolk County, NY",US,40.9849,-72.6151,2020-03-16,0,0,0,False,Suffolk County,New York,0,0
